<a href="https://colab.research.google.com/github/kch34/nlp2021_hw1/blob/main/CS2731_Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Document Classification

This notebook is designed to get you started thinking about how to use notebooks in a more systematic way as you design machine learning programs for text.  

Along the way, we'll see some cool bits of python that you might not have thought of (or known to google for) that make many of the things we'll want to do very easy.  And it includes [some design patterns][1] for exploring representation changes in classification problems that you can work with as you push your ideas forward.

[1]:https://en.wikipedia.org/wiki/Software_design_pattern

In [ ]:
from __future__ import print_function
import os
import nltk
import re
import itertools
import vocabulary
import newsreader
import numpy as np
import scipy
import sklearn
from sklearn.preprocessing import StandardScaler
try:
    import cPickle as pickle
except:
    import pickle

## 0. An introduction to using notebooks for research

### Design Pattern 0.0: Keep the things you have to customize separate

It's always easy to get carried away getting settled into a particular setup and working on a particular experiment.  But the first step in making your work general is [DRY: don't repeat yourself][1].  That means setting up variables once and for all for the things you're going to have to get right to work with particular data in a particular environment.  Let's get organized.

[1]:https://en.wikipedia.org/wiki/Don't_repeat_yourself

In [ ]:
vocab_file, vocab_file_type = "reviews-vocab.pkl", "pickle"

embedding_file, embedding_dimensions, embedding_cache = \
    "glove.6B.50d.txt", 50, "reviews-embedding.npz"

all_data, train_dir, dev_dir, test_dir = \
    "reviews", "reviews/train/", "reviews/dev/", "reviews/test/"
    
class1, class2 = "pos", "neg"

has_bad_metadata = False

### Design Pattern 0.1: Save your work

You won't always be able to keep your notebook running indefinitely.  (For one thing, if you are updating basic aspects of the software you're using, the convenions of the python import method means that you'll have to restart the python kernel in order to load the updates.)  On the other hand, some of the computations you want to exploit in dealing with text data are expensive.  So consider using the [python pickle serialization framework][1] and [loading and saving functionality from numpy][2] in cases like this to let you pick up right where you left off.

Any time you change the overall parameters of the learning algorithm, set `made_vocabulary` and `made_embedding` to `False`, to reindex the words and get a general word similarity matrix for them.  Once you've done this, set `made_vocabulary` and `made_embedding` to `True`, so you simply load the precomputed representations from the file system. 

[1]:http://www.ibm.com/developerworks/library/l-pypers/index.html
[2]:https://docs.scipy.org/doc/numpy/reference/generated/numpy.load.html

In [ ]:
made_vocabulary = True
if made_vocabulary :
    v = vocabulary.Vocabulary.load(vocab_file, file_type=vocab_file_type)
else: 
    tokens = newsreader.all_textfile_tokens(all_data, strip_metadata=has_bad_metadata)                                            
    v = vocabulary.Vocabulary.from_iterable(tokens, file_type=vocab_file_type)
    v.save(vocab_file)
v.stop_growth()

In [ ]:
made_embedding = True
if made_embedding :
    e = newsreader.load_sparse_csr(embedding_cache)
else: 
    e = newsreader.build_sparse_embedding(v, embedding_file, embedding_dimensions)
    newsreader.save_sparse_csr(embedding_cache, e)

### Design Pattern 0.2: Design your notebook to evolve

A notebook is fundamentally a record that you are using to document your work and maintain the rationale for particular programming decisions.  That means you're going to be adapting it.  You want it to be easy to make changes, and you also want it to be possible to read an updated version and understand what's going on.  At the same time, you also want to be able to focus your work on the particular experiments you're exploring at a particular stage in your work, without losing the effort that you've done earlier.

Here are some suggestions and infrastructure to make this easier.
- Names.  In practice, it's too much effort to give every experiment you do a descriptive name.  Ultimately you have to look at the specific parameters and setup of the experiment anyway, to draw conclusions.  So you're going to wind up using arbitrary elements to distinguish your experiments.  Use names that don't assume a particular order, so you can add new names anywhere, and use names that indicate the dependencies on earlier computations, so you can select specific subparts of the notebook to run.
- Sections.  Divide your experiments into sections and use this organization to explain the thinking that you're doing and to find results that you want to go back to.  Consider including the section identifiers strategically in the writeup to help contextualize the documents (for example, the way I'm doing with the design patterns here.)
- Limit computations.  The code below allows you to restrict computation to expressions that contribute to the definitions of particular target variables, so you can rerun the entire notebook but just focus on the experiments that you are currently working on.

In [ ]:
targets = []
def selected(name) :
    if not targets:
        return True
    if any(t.startswith(name) for t in targets) :
        return True
    return False

### Example 0.0: Finding features

You're going to want to use the data manager class, which has placeholders that say how you might build a feature dictionary by using the training data and then how you will scan files to create feature matrices from file tokens.  Here are default functions that carry out those operations in the simplest way possible.  **Remember: simple is good.** Being fancy is just as likely to get you into trouble as it is to help you.  If you have these functions defined, you can make a data manager with a call like this (where `v` is your vocabulary)

```python
my_data = newsreader.DataManager(class1_training_directory,
                                 class2_training_directory,
                                 class1_test_directory,
                                 class2_test_directory,
                                 use_default_features(v), 
                                 count_features)
```

Note that this template is set up for 20 newsgroups classification problems, where you're not given an explicit development set to work with, and you'll want to strip the metadata from the files before processing them.  We do something slightly different below for the reviews dataset, since development data is given and there's no metadata to strip from the files. 
                                 

In [ ]:
def use_default_features(vocab) :
    return lambda data: vocab

def count_features(features, gen_tokens) :
    for t in gen_tokens :
        r = features.add(t)
        if r :
            yield r    
            
count_data = newsreader.DataManager(train_dir + class1,
                                       train_dir + class2,
                                       test_dir + class1,
                                       test_dir + class2,
                                       use_default_features(v),
                                       count_features,
                                       dev_dir + class1 if dev_dir else None,
                                       dev_dir + class2 if dev_dir else None,
                                       strip_metadata=has_bad_metadata)

count_data.initialize(build_cache=True)

## 1. Representing the Data

### Design Pattern 1.0: Higher-order functions for one-off abstraction

We're going to be using `scipy` sparse matrices to handle language data (as we must), but unfortunately it turns out that a lot of the nice operations that you can do on `numpy` matrices don't work on sparse matrices, because they wouldn't preserve the sparsity in the general case.  One of the things that turns out to be impossible is to "forget" the  values in a matrix and replace all the nonzero values in a matrix with a sensible value like 1.  Bottom line: if we want to have yes/no features in our data matrices, rather than count features, we're going to have to remove redundancies right away when we scan the file.

Now, if you think for a second, you'll realize that this effect doesn't have anything to do with what features we're looking for or how we're looking for them.  Features are always integers, and the way we merge redundancies is to accumulate the features that we've seen so far into a set and, once we're done scanning a file, we then emit the features we observed.

Lots of programming languages (but maybe not the ones that you're used to) let you just parameterize this operation by the basic `feature_counter` operation, and create abstract, general code without the nuisance of making objects, classes, data types or other heavy-duty discipline.  Here's how you do it in python.  As it happens, I wanted to experiment with boolean features in exploring the design space for text classifiers today, so this is what I will use.

In [ ]:
def make_boolean_features(feature_counter) :
    def collect_features(features, gen_tokens) :
        seen = set()
        for f in feature_counter(features, gen_tokens) :
            seen.add(f)
        for f in seen :
            yield f
    return collect_features

boolean_data = newsreader.DataManager(train_dir + class1,
                                         train_dir + class2,
                                         test_dir + class1,
                                         test_dir + class2,
                                         use_default_features(v),
                                         make_boolean_features(count_features),
                                         dev_dir + class1 if dev_dir else None,
                                         dev_dir + class2 if dev_dir else None,
                                         strip_metadata=has_bad_metadata)

boolean_data.initialize(build_cache=True)

### Design Pattern 1.1: Bundle your data and provide consistent views

From data management, we've learned that analysts are often interested in [views][1], which are systematically related to underlying data but record the results of inferences and other computations which are precomputed so that you can have fast access to them.

This is going to be important as you explore the best ways  to represent the underlying text, and design experiments that are going to compare these representations to one another.  

This is the kind of thing that it's nice to formalize with a python class.  The `Experiment` class below sets up the problem of learning a classifier from a data manager, arranges the data acquisition and classifier training, and carries out a preliminary validation.  Most importantly, it provides a general method for taking your data, rerepresenting it in a consistent way, and setting up another experiment as a result.

### Design Pattern 1.2: But keep the test data sacred

**You don't want to make it easy to peek at the test data, which is special and limited.** So even though this class keeps the structure in place so that you manage a consistent representation of the test data, you have to add code to evaluate on the test data yourself.


[1]:https://en.wikipedia.org/wiki/View_(SQL)

In [ ]:
class Experiment(object) :
    '''Organize the process of getting data, building a classifier,
    and exploring new representations'''
    
    def __init__(self, data, comment, classifier, cdesc) :
        'set up the problem of learning a classifier from a data manager'
        self.data = data
        self.comment = comment
        self.classifier = classifier
        self.cdesc = cdesc
        self.initialized = False
        
    def initialize(self) :
        'materialize the training data, dev data and test data as matrices'
        if not self.initialized :
            self.train_X, self.train_y = self.data.training_data()
            self.dev_X, self.dev_y = self.data.dev_data()
            self.test_X, self.test_y = self.data.test_data()
            self.initialized = True
        
    def fit_and_validate(self, report=True) :
        'train the classifier and assess predictions on dev data'
        if not self.initialized :
            self.initialize()
        self.classifier.fit(self.train_X, self.train_y)
        self.dev_predictions = self.classifier.predict(self.dev_X)
        self.accuracy = sklearn.metrics.accuracy_score(self.dev_y, self.dev_predictions)
        if report :
            print("{}\nclassified by {}\naccuracy {}".format(self.comment, self.cdesc, self.accuracy))
            
    def xval(self, folds=20, report=True) :
        accuracies = []
        for i in range(folds) :
            self.fit_and_validate(report=False)
            accuracies.append(self.accuracy)
        if report :
            msg = "{}\nclassified by {}\naverage accuracy {} (std {})"
            print(msg.format(self.comment, self.cdesc, 
                             sum(accuracies)/folds,
                             np.std(accuracies)))
    
    @classmethod
    def transform(cls, expt, operation, description, classifier, cdesc) :
        'use operation to transform the data from expt and set up new classifier'
        if not expt.initialized :
            expt.initialize()
        result = cls(expt.data, expt.comment + '\n' + description, classifier, cdesc)
        result.train_X, result.train_y = operation(expt.train_X, expt.train_y, 'train')
        result.dev_X, result.dev_y = operation(expt.dev_X, expt.dev_y, 'dev')
        result.test_X, result.test_y = operation(expt.test_X, expt.test_y, 'test')
        result.initialized = True
        return result
        

### Example 1.0: Setting up an experiment

Pretty straightforward.

In [ ]:
if selected("expt_10_"):
    expt_10_ = Experiment(count_data,
                       "{}: {} vs {}, using word count features".format(all_data, class1, class2),
                       sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                       "logistic regression")
    expt_10_.initialize()
    expt_10_.xval()

In [ ]:
if selected("expt_11_") :
    expt_11_ = Experiment(boolean_data,
                         "{}: {} vs {}, using word presence/absence features".format(all_data, class1, class2),
                         sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                         "logistic regression")
    expt_11_.initialize()
    expt_11_.xval()

## 2. A Guide to Text Classification

### Example 2.0: Making feature values meaningful.

In preparing this assignment, I read [an ACL 2012 paper by Sida Wang and Chris Manning][2] about text classification.  They suggested weighting word features not based on counts, or 1-0, or based on TF-IDF, but instead based on the evidence that the features provide for the eventual class.  This is a cool and influential idea.  All you need to implement it is to work with binary features, compute a score for each feature, and then reweight by the scores.  It's crazy simple code.
* Use boolean indexing and slices to get the data
* Sparse matrices have built-in methods for counting nonzero elements
* Numpy lets you compute the relevant operations elementwise over the matrix.

[2]:http://aclweb.org/anthology/P/P12/P12-2018.pdf

In [ ]:
def wang_manning_weights(expt) :
    Xyes = expt.train_X[expt.train_y ==1, :]
    Xno = expt.train_X[expt.train_y != 1, :] 
    yesrates = np.log((Xyes.getnnz(axis=0) + 1.) / Xyes.shape[1])
    norates = np.log((Xno.getnnz(axis=0) + 1.) / Xno.shape[1])
    W = scipy.sparse.diags(yesrates - norates, 0)
    return lambda X, y, c: (X.dot(W), y)

The infrastructure that we have in place already to work with data makes this super-easy to try out.

In [ ]:
if selected("expt_11_20_") :
    expt_11_20_ = Experiment.transform(expt_11_,
                             wang_manning_weights(expt_11_),
                             "features weighted by evidence they give of class",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                             "logistic regression")
    expt_11_20_.xval()

This is not just a one-off pattern for this particular research result.  For example, if you don't bother to separate the positive and negative examples, and add a call to [np.reciprocal][1], you'll basically turn this into TFIDF weighting.

[1]:https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.reciprocal.html

In [ ]:
def idf_weights(expt) :
    idf = np.log((expt.train_X.shape[1] + 1.) / (expt.train_X.getnnz(axis=0) + 1.))
    W = scipy.sparse.diags(idf, 0)
    return lambda X, y, c: (X.dot(W), y)

In [ ]:
if selected("expt_10_20_") :
    expt_10_20_ = Experiment.transform(expt_10_,
                             idf_weights(expt_10_),
                             "features weighted by inverse document frequency",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                             "logistic regression")
    expt_10_20_.xval()

### Example 2.1: Drawing on additional resources

One of the important ways to improve results on NLP problems is to get access to additional resources.  You always have a limited amount of training data for the problem you're interested in, which will include instances with properties that you have seen rarely if at all.  If you have some general sense of how words behave, from text corpora or dictionaries, you can use that information to generalize from words that you have seen to words that you haven't.

Word embeddings, of course, are a prototypical example of this.  To use them, you just add features to your data points computed from the word embeddings.  The sparse matrix `stack` operations make it easy to construct the matrices you need. (Note: if you have extra features that aren't in the word embedding, e.g., bigrams, then you need to account for that in applying the generic word embeddings; conversely, you probably want to add the word embedding features to your problem, since you may actually have information about particular words that gives special but reliable information about your classification problem that you do not want to discard.)  With these hints, you can probably understand the code for adding embedding information to your classification problem.

Another thing you could try along these lines would be to build an embedding matrix by applying SVD to the training data, as in the latent semantic analysis exercise we did two weeks ago.

```python
_, _, wrt = scipy.sparse.linalg.svds(expt.X_train, k=r)
W = np.transpose(wrt))
```

In [ ]:
def add_embeddings(expt, embeddings, scale=True, stack=True) :
    extra_features = expt.train_X.shape[1] - embeddings.shape[0]
    if extra_features > 0 :
        Z = scipy.sparse.csr_matrix((extra_features, embeddings.shape[1]))
        W = scipy.sparse.vstack([embeddings, Z])
    else: 
        W = embeddings
    if scale :
        scaler = StandardScaler(with_mean=False)
        scaler.fit(expt.train_X.dot(W))
    def operation(X, y, s) :
        if scale:
            new_features = scaler.transform(X.dot(W))
        else :
            new_features = X.dot(W)
        if stack :
            all_features = scipy.sparse.hstack([X, new_features]).tocsr()
        else :
            all_features = new_features
        return (all_features, y)
    return operation

def dimensionality_reduction(expt, dimensions) :
    _, _, wrt = scipy.sparse.linalg.svds(expt.train_X, k=dimensions, 
                                         return_singular_vectors='vh')
    return add_embeddings(expt, np.transpose(wrt), stack=False)

Here are a few examples: using word embeddings and doing LSA.

In [ ]:
if selected("expt_10_211_") :
    expt_10_211_ = Experiment.transform(expt_10_,
                             add_embeddings(expt_10_, e),
                             "enriched via word embeddings",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_10_211_.xval()

if selected ("expt_10_212_") :
    expt_10_212_ = Experiment.transform(expt_10_,
                             dimensionality_reduction(expt_10_, 100),
                             "transformed via LSA(100)",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                           "logistic regression")
    expt_10_212_.xval()
    
if selected("expt_11_211_") :
    expt_11_211_ = Experiment.transform(expt_11_,
                             add_embeddings(expt_11_, e),
                             "enriched via word embeddings",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_11_211_.xval()
 
if selected("expt_11_20_211_") :
    expt_11_20_211_ = Experiment.transform(expt_11_20_,
                             add_embeddings(expt_11_20_, e),
                             "enriched via word embeddings",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_11_20_211_.xval()


### Example 2.2: Fancy feature definitions

The place that you get a payoff from a good design is when you're able to easily try something out that you didn't anticipate when you started writing the code.  Invariably, that's a bit lucky when it works out, and anything that you start pushing too hard in a direction that it wasn't designed for becomes more and more cumbersome until you can't stand it.  But we're not there yet in this assignment.

Last time we looked at [nltk's resources for analyzing collocations][1], because, as we saw, it's an interesting unsupervised technique for characterizing word senses and dealing with ambiguous words.  The architecture that we have for identifying features in the training data makes it possible to preprocess the training data to look for meaningful bigram features.  (The strategy I've used here is not exactly the one that [Wang and Manning][2] recommend by the way!)  Python generator functions make it straightforward to report instances of these features in the course of scanning through a file.  

So it's totally routine to try this out by adapting the work we've done so far.  Who knows what else we could handle this way!

[1]:http://www.nltk.org/howto/collocations.html
[2]:http://aclweb.org/anthology/P/P12/P12-2018.pdf

In [ ]:
def use_bigram_features(data) :
    f = vocabulary.Vocabulary.load(vocab_file, file_type=vocab_file_type)
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    word_fd = nltk.FreqDist(data.all_train_tokens())
    bigram_fd = nltk.FreqDist(nltk.bigrams(data.all_train_tokens()))
    finder = nltk.collocations.BigramCollocationFinder(word_fd, bigram_fd)
    finder.apply_freq_filter(5)
    collocations = filter(lambda (g,i): i > 0, finder.score_ngrams(bigram_measures.pmi))
    for (w1, w2), _ in collocations:
        f.add(w1 + " " + w2)
    f.stop_growth()
    return f

def count_bigram_features(features, gen_tokens) :
    prev = None
    for t in gen_tokens :
        r = features.add(t)
        if r :
            yield r
            if prev :
                r = features.add(prev + " " + t)
                if r : 
                    yield r
            prev = t

In order to test this, we first need to reload the data with the new feature definitions.

In [ ]:
bigram_data = newsreader.DataManager(train_dir + class1,
                                       train_dir + class2,
                                       test_dir + class1,
                                       test_dir + class2,
                                       use_bigram_features,
                                       make_boolean_features(count_bigram_features),
                                       dev_dir + class1 if dev_dir else None,
                                       dev_dir + class2 if dev_dir else None,
                                       strip_metadata=has_bad_metadata)

bigram_data.initialize(build_cache=True)

As before, we create a basic experiment to build a classifier.

In [ ]:
if selected("expt_22_") :
    expt_22_ = Experiment(bigram_data,
                   "{}: {} vs {}, using word and bigram presence/absence features".format(all_data, class1, class2),
                   sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                   "logistic regression")
    expt_22_.initialize()
    expt_22_.xval()

if selected("expt_22_220_") :
    expt_22_220_ = Experiment.transform(expt_22_,
                             wang_manning_weights(expt_22_),
                             "features weighted by evidence they give of class",
                             sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                             "logistic regression")
    expt_22_220_.initialize()
    expt_22_220_.xval()


We can compare the basic version to a version with embeddings.

In [ ]:
if selected("expt_22_221_") :
    expt_22_221_ = Experiment.transform(expt_22_,
                             add_embeddings(expt_22_, e),
                             "with word embedding features added",
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_22_221_.xval()

if selected("expt_22_220_221_") :
    expt_22_220_221_ = Experiment.transform(expt_22_220_,
                             add_embeddings(expt_22_220_, e),
                             "with word embedding features added",
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_22_220_221_.xval()


### Example 2.3 Understanding the learning curve

Different feature representations may work better depending on the amount of training data available.  The transformation setup we have makes it easy to consider just a small amount of training data -- by selecting a random subset of the training data.

In [ ]:
def limit_training(percent) :
    def operation(X, y, s) :
        if s != 'train' :
            return (X, y)
        data_to_take = int(X.shape[0] * percent)
        indices = np.random.choice(X.shape[0], 
                                      size=data_to_take,
                                      replace=False)
        return (X[indices,:], y[indices])
    return operation

Here's an illustration -- how much do the embeddings help with only 10% of the training data.  One issue is that the fitting is very noisy!

In [ ]:
if selected("expt_11_231_") :
    expt_11_231_ = Experiment.transform(expt_11_,
                             limit_training(0.1),
                             "considering 10% training data",
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_11_231_.xval()

if selected("expt_11_211_232_") :
    expt_11_231_232_ = Experiment.transform(expt_11_211_,
                             limit_training(0.1),
                             "considering 10% training data",
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_11_231_232_.xval()


if selected("expt_11_20_233_") :
    expt_11_20_232_ = Experiment.transform(expt_11_20_,
                             limit_training(0.1),
                             "considering 10% training data",
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_11_20_232_.xval()

if selected("expt_11_20_211_234_") :
    expt_11_20_211_232_ = Experiment.transform(expt_11_20_211_,
                             limit_training(0.1),
                             "considering 10% training data",
                            sklearn.linear_model.SGDClassifier(loss="log",
                                       penalty="elasticnet",
                                       n_iter=50),
                            "logistic regression")
    expt_11_20_211_232_.xval()
